In [45]:
!pip install -U kaleido
import plotly.io as pio

In [46]:
import pandas as pd
import plotly.graph_objects as go
import datetime
from datetime import date

In [47]:
ids = [
    "379.4_ING_CORR_1006__18_9", # Ingresos Corrientes 1993-2006
    "379.5_ING_CORR_2014__18_29", # Ingresos Corrientes 2007-2014
    "379.6_ING_CORR_2017__13_43", # Ingresos Corrientes 2015-

    "379.4_GTOS_CORR_006__19_2", # Gastos Corrientes 1993-2006
    "379.5_GTOS_CORR_014__19_73", # Gastos Corrientes 2007-2014
    "379.6_GTOS_CORR_017__14_17", # Gastos Corrientes 2015-

    "379.4_RECURSOS_C006__41_27", # Recursos de capital 1993-2006
    "379.5_RECURSOS_C014__41_42", # Recursos de capital 2007-2014
    "379.6_RECURSOS_C017__36_95", # Recursos de capital 2015-

    "379.4_GTOS_CAP_1006__18_35", # Gastos de capital 1993-2006
    "379.5_GTOS_CAP_2014__18_77", # Gastos de capital 2007-2014
    "379.6_GTOS_CAP_2017__13_95", # Gastos de capital 2015-

    "379.4_SUPERAVIT_006__28_94", # Resultado Primario 1993-2006
    "379.5_SUPERAVIT_014__28_85", # Resultado Primario 2007-2014
    "379.6_SUPERAVIT_017__23_1",  # Resultado Primario 2015-

    "379.4_RESULTADO_006__46_38", # Resultado Económico 1993-2006
    "379.5_RESULTADO_014__46_23", # Resultado Económico 2007-2014
    "379.6_RESULTADO_017__41_16", # Resultado Económico 2015-

    "379.4_RESULTADO_006__23_0",  # Resultado Financiero 1993-2006
    "379.5_RESULTADO_014__23_60", # Resultado Financiero 2007-2014
    "379.6_RESULTADO_017__18_10", # Resultado Financiero 2015-

    "379.4_GTOS_CORR_006_II_0_T_49_90", # Intereses deuda externa 1993-2006
    "379.4_GTOS_CORR_006__49_73", # Intereses deuda interna 1993-2006
    "379.5_GTOS_CORR_014_1_0_T_49_78", # Intereses moneda extranjera 2007-2014
    "379.5_GTOS_CORR_014__47_41", # Interes moneda local 2007-2014
    "379.6_GTOS_CORR_017__40_26", # Intereses Netos 2015-

    "9.2_PPC_2004_T_22", # PIB
       ]

In [48]:
ids_str = ",".join(ids)

In [49]:
url_df = f"http://apis.datos.gob.ar/series/api/series/?ids={ids_str}&limit=1000&format=csv"

In [50]:
df = pd.read_csv(url_df, storage_options = {'User-Agent': 'Mozilla/5.0'})

In [51]:
subcols = [df.columns[i:i+3] for i in range(1, len(df.columns[0:22]), 3)] + [df.columns[j:j+5] for j in range(22, 22+len(df.columns[22:22+5]), 5)]

In [52]:
for subcol, item in zip(
    subcols,
    ["ing_corr", "g_corr", "rec_k", "g_k", "r_prim", "r_ec", "r_fin", "int_net"]):
  df[item] = df[subcol].sum(axis=1)
  df[f"{item}_gdp"] = df[item] / (df["pib_precios_corrientes"] / 4) * 100

In [53]:
df["bcra_gdp"] = df[df.columns[27]] / (df["pib_precios_corrientes"] / 4) * 100

In [54]:
df = df[[col for col in df.columns if "_gdp" in col or col == "indice_tiempo"]]

In [55]:
df = df.dropna(subset=df.columns[1:8].tolist())

In [56]:
df = df[12:].reset_index(drop=True)

In [57]:
df["indice_tiempo"] = pd.to_datetime(df["indice_tiempo"])

In [58]:
df["r_k_gdp"] = df["rec_k_gdp"] - df["g_k_gdp"]

In [59]:
def graph_1(df, variables, color_list, names, lines, fig):
  for trace, color, name in zip(
      variables,
      color_list,
      names):
    if trace in lines:
      fig.add_trace(
        go.Scatter(
          x=df["indice_tiempo"].dt.strftime('%Y-%m'),
          y=df[trace],
          mode='lines+markers',
          marker=dict(color=color, size=6),
          line=dict(color=color, width=2),
          name=name
        )
      )
    else:
      fig.add_trace(
        go.Bar(
          x=df["indice_tiempo"].dt.strftime('%Y-%m'),
          y=df[trace],
          marker=dict(color=color),
          name=name
        )
      )

In [60]:
def update_layout_fig(fig, min, max, yaxis_title, title_text):
  fig.update_layout(
    margin=dict(
      t = 125,
      b = 50,
      r = 25,
      l = 25
  ),
    height=850,
    width=850,
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        orientation='h',
        xanchor='center',
        x=0.5,
        yanchor='bottom',
        y=1,
        font=dict(size=16)
        ),
    xaxis=dict(
        range=[df["indice_tiempo"][0]-datetime.timedelta(days=30),
               df["indice_tiempo"].iloc[-1]+datetime.timedelta(days=30)],
        title = None,
        ticklabelmode="period",
        dtick="M6",
        tickformat="Q%q\n<b>%Y",
        tickangle = 0
      ),
    yaxis=dict(
        range=[min, max],
        dtick=2.5,
        title=dict(text=yaxis_title),
        showgrid=True,
        gridcolor='lightgray'
        ),
    font=dict(
        size=14,
        family="Serif",
        ),
    title = dict(
    text = title_text,
    xanchor = 'left',
    x = 0.0325,
    yanchor = 'top',
    y = 0.95,
    font = dict(
        size = 24,
        color = 'black'
    )
  ),
  )

  fig.update_yaxes(
      zeroline=True,
      zerolinecolor='black',
      zerolinewidth=1
  )

In [61]:
fig_1 = go.Figure()

graph_1(df,
        [df.columns[5], df.columns[7], df.columns[8]],
        ["#87ceeb", "#191970", "#ffdbe2"],
        ["Primary balance", "Fiscal balance", "Net interest payments"],
        [df.columns[5], df.columns[7]],
        fig_1)

update_layout_fig(fig_1,
                  -15,
                  5,
                  "<b>% GDP",
                  "<b>Argentina's fiscal performance</b><br><sup>Primary & fiscal balance and net interest payments.<br><sup>Quarterly data from 2007Q1 to 2024Q2. <b>Source</b>: ONP & INDEC. </sup></sup>")

In [62]:
fig_1.show()

In [63]:
pio.write_html(fig_1,
               file='fiscal_1.html',
               full_html=True)

In [64]:
fig_2 = go.Figure()

graph_1(df,
        [df.columns[6], df.columns[10], df.columns[7]],
        ["#fa8072", "#40e0d0", "#ffdab9"],
        ["Economic result", "Capital result", "Fiscal balance"],
        [df.columns[6], df.columns[10]],
        fig_2)
update_layout_fig(fig_2,
                  -15,
                  5,
                  "<b>% GDP",
                  "<b>Fiscal balance in Argentina: behind-the-scenes</b><br><sup>Main components of the fiscal balance.<br><sup>Quarterly data from 2007Q1 to 2024Q2. <b>Source</b>: ONP & INDEC.</sup></sup>")

In [65]:
fig_2.show()

In [66]:
pio.write_html(fig_2,
               file='fiscal_2.html',
               full_html=True)